<a href="https://colab.research.google.com/github/DESOLATE17/ML/blob/main/RK2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ИУ5-64Б Такташова Дарья, Вариант 17

Датасет: https://www.kaggle.com/mathan/fifa-2018-match-statistics

**Метод №1**: Линейная/логистическая регрессия


**Метод №2**: Градиентный бустинг


**Задание.** Для заданного набора данных (по Вашему варианту) постройте модели классификации или регрессии (в зависимости от конкретной задачи, рассматриваемой в наборе данных). Для построения моделей используйте методы 1 и 2 (по варианту для Вашей группы). Оцените качество моделей на основе подходящих метрик качества (не менее двух метрик). Какие метрики качества Вы использовали и почему? Какие выводы Вы можете сделать о качестве построенных моделей? Для построения моделей необходимо выполнить требуемую предобработку данных: заполнение пропусков, кодирование категориальных признаков, и т.д.

In [12]:
import numpy as np
import pandas as pd

df = pd.read_csv("/content/FIFA 2018 Statistics.csv")
df

,Date,Team,Opponent,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,...,Yellow Card,Yellow & Red,Red,Man of the Match,1st Goal,Round,PSO,Goals in PSO,Own goals,Own goal Time
0,14-06-2018,Russia,Saudi Arabia,5,40,13,7,3,3,6,...,0,0,0,Yes,12.0,Group Stage,No,0,NaN,NaN
1,14-06-2018,Saudi Arabia,Russia,0,60,6,0,3,3,2,...,0,0,0,No,NaN,Group Stage,No,0,NaN,NaN
2,15-06-2018,Egypt,Uruguay,0,43,8,3,3,2,0,...,2,0,0,No,NaN,Group Stage,No,0,NaN,NaN
3,15-06-2018,Uruguay,Egypt,1,57,14,4,6,4,5,...,0,0,0,Yes,89.0,Group Stage,No,0,NaN,NaN
4,15-06-2018,Morocco,Iran,0,64,13,3,6,4,5,...,1,0,0,No,NaN,Group Stage,No,0,1.0,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,11-07-2018,England,Croatia,1,46,11,1,6,4,4,...,1,0,0,No,5.0,Semi- Finals,No,0,NaN,NaN
124,14-07-2018,Belgium,England,2,43,12,4,3,5,4,...,1,0,0,Yes,4.0,3rd Place,No,0,NaN,NaN
125,14-07-2018,England,Belgium,0,57,15,5,7,3,5,...,2,0,0,No,NaN,3rd Place,No,0,NaN,NaN
126,15-07-2018,France,Croatia,4,39,8,6,1,1,2,...,2,0,0,Yes,18.0,Final,No,0,1.0,18.0


In [13]:
df.isnull().sum()

Date                        0
Team                        0
Opponent                    0
Goal Scored                 0
Ball Possession %           0
Attempts                    0
On-Target                   0
Off-Target                  0
Blocked                     0
Corners                     0
Offsides                    0
Free Kicks                  0
Saves                       0
Pass Accuracy %             0
Passes                      0
Distance Covered (Kms)      0
Fouls Committed             0
Yellow Card                 0
Yellow & Red                0
Red                         0
Man of the Match            0
1st Goal                   34
Round                       0
PSO                         0
Goals in PSO                0
Own goals                 116
Own goal Time             116
dtype: int64

In [14]:
df = df.drop(['Date','Own goals','Own goal Time'],axis=1)
df['1st Goal'] = df['1st Goal'].fillna(df['1st Goal'].mean())

Закодируем категориальные признаки

In [15]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

df['Man of the Match'] = le.fit_transform(df['Man of the Match'])
motm_map = {i:l for i,l in enumerate(le.classes_)}
motm_map

{0: 'No', 1: 'Yes'}

In [16]:
df['PSO'] = le.fit_transform(df['PSO'])
pso_map = {i:l for i,l in enumerate(le.classes_)}
pso_map

{0: 'No', 1: 'Yes'}

In [17]:
rounds = list(df['Round'].unique())
rounds

['Group Stage',
 'Round of 16',
 'Quarter Finals',
 'Semi- Finals',
 '3rd Place',
 'Final']

In [18]:
rounds_map = {l:i for i,l  in enumerate(rounds)}
rounds_map

{'Group Stage': 0,
 'Round of 16': 1,
 'Quarter Finals': 2,
 'Semi- Finals': 3,
 '3rd Place': 4,
 'Final': 5}

In [19]:
df['Round'] = df['Round'].apply(lambda x: rounds_map[x])

In [20]:
df = df.drop(['Team','Opponent'],axis=1)

In [21]:
df

,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,Offsides,Free Kicks,Saves,...,Distance Covered (Kms),Fouls Committed,Yellow Card,Yellow & Red,Red,Man of the Match,1st Goal,Round,PSO,Goals in PSO
0,5,40,13,7,3,3,6,3,11,0,...,118,22,0,0,0,1,12.000000,0,0,0
1,0,60,6,0,3,3,2,1,25,2,...,105,10,0,0,0,0,39.457447,0,0,0
2,0,43,8,3,3,2,0,1,7,3,...,112,12,2,0,0,0,39.457447,0,0,0
3,1,57,14,4,6,4,5,1,13,3,...,111,6,0,0,0,1,89.000000,0,0,0
4,0,64,13,3,6,4,5,0,14,2,...,101,22,1,0,0,0,39.457447,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,1,46,11,1,6,4,4,3,24,5,...,148,14,1,0,0,0,5.000000,3,0,0
124,2,43,12,4,3,5,4,1,5,5,...,108,11,1,0,0,1,4.000000,4,0,0
125,0,57,15,5,7,3,5,0,12,2,...,110,5,2,0,0,0,39.457447,4,0,0
126,4,39,8,6,1,1,2,1,14,1,...,99,14,2,0,0,1,18.000000,5,0,0


In [22]:
X = df.drop(['Man of the Match'],axis=1)
y = df['Man of the Match']

In [23]:
from sklearn.preprocessing import RobustScaler

sc = RobustScaler()
X = pd.DataFrame(sc.fit_transform(X),columns=X.columns)

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

Для оценки качества модели линейной регрессии и модели градиентного бустинга, мы будем использовать метрики, такие как среднеквадратичная ошибка (MSE), коэффициент детерминации (R^2) и средняя абсолютная ошибка (MAE). Эти метрики помогут нам понять, насколько хорошо каждая модель выполняет прогнозирование.

In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_lr = lr.predict(X_test)

# Расчет метрик
mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)
mae_lr = mean_absolute_error(y_test, y_pred_lr)

print(f"MSE: {mse_lr:.2f}")
print(f"R^2: {r2_lr:.2f}")
print(f"MAE: {mae_lr:.2f}")

MSE: 0.21
R^2: 0.16
MAE: 0.36


In [26]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gbr.fit(X_train, y_train)

y_pred_gbr = gbr.predict(X_test)

mse_gbr = mean_squared_error(y_test, y_pred_gbr)
r2_gbr = r2_score(y_test, y_pred_gbr)
mae_gbr = mean_absolute_error(y_test, y_pred_gbr)

print(f"GBR MSE: {mse_gbr:.2f}")
print(f"GBR R^2: {r2_gbr:.2f}")
print(f"GBR MAE: {mae_gbr:.2f}")

GBR MSE: 0.23
GBR R^2: 0.06
GBR MAE: 0.42


Модель градиентного бустинга показывает более низкие значения MSE и MAE, а также выше значение (R^2) по сравнению с линейной моделью, что указывает на более высокую эффективность модели градиентного бустинга в предсказательном плане.
Линейная модель демонстрирует значительно худшие результаты, что может указывать на то, что данные содержат сложные зависимости, которые не могут быть полностью учтены линейной моделью.
Выбор модели должен основываться на специфике задачи и целях исследования. В данном случае, модель градиентного бустинга кажется более подходящей для этой задачи благодаря более низким значениям ошибок и лучшей способности объяснять вариацию в данных.